# Load packages

In [8]:
%load_ext autoreload
%autoreload 2
from covid_constants_and_util import *
import helper_methods_for_aggregate_data_analysis as helper
import pandas as pd
import geopandas as gpd
import os
import matplotlib.pyplot as plt
import json
import datetime
import scipy
import time
import glob

JUST_TESTING = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Understand the Safegraph data from API

In [9]:
csv_dir = r'/media/gpu/Seagate/SC_weekly_patterns_20220101'

csv_parts = glob.glob(os.path.join(csv_dir, "SC-part*.csv.gz"))

csv_parts

['/media/gpu/Seagate/SC_weekly_patterns_20220101/SC-part1.csv.gz',
 '/media/gpu/Seagate/SC_weekly_patterns_20220101/SC-part2.csv.gz',
 '/media/gpu/Seagate/SC_weekly_patterns_20220101/SC-part3.csv.gz',
 '/media/gpu/Seagate/SC_weekly_patterns_20220101/SC-part4.csv.gz',
 '/media/gpu/Seagate/SC_weekly_patterns_20220101/SC-part5.csv.gz',
 '/media/gpu/Seagate/SC_weekly_patterns_20220101/SC-part6.csv.gz']

In [ ]:
df = pd.read_csv(csv_parts[0])

df

## split according to the cbg

In [32]:
df['county_fips'] = df['sg_wp__poi_cbg'].astype(str).str.zfill(5).str[:6]
df['state_fips'] = df['sg_wp__poi_cbg'].astype(str).str.zfill(5).str[:2]
df['date_range'] = df['date_range_start'] + "_" + df['date_range_end']


In [34]:
df_gb = df.groupby(['state_fips', 'county_fips', 'date_range'])

In [56]:
def split_a_part(df, save_path, is_append=False, is_compressed=False):    
    suffix = 'csv'
    if is_compressed:
        suffix = 'csv.gz'

    cnt = 0
    for i, x in df_gb:
        # get data
        state_fips = x.iloc[0]['state_fips']
        county_fips = x.iloc[0]['county_fips']
        date_range = x.iloc[0]['date_range']

        # generate new file name
        folder = os.path.join(save_path, state_fips, county_fips)
        os.makedirs(folder, exist_ok=True)
        base_name = f"{county_fips}_{date_range}.{suffix}"
        new_name = os.path.join(folder, base_name)    

        if is_append:

            if not os.path.exists(new_name):
                x.to_csv(new_name, index=False)
            else:
                x.to_csv(new_name, index=False, mode='a', header=None)    
        else:
            x.to_csv(new_name, index=False)        

        x.to_csv()
        cnt += 1
        if cnt % 500 == 0:
            print(cnt, i, folder)
            
def split_all_part(csv_parts, save_path, is_append=False, is_compressed=False):
    for idx, csv_part in enumerate(csv_parts):
        print(f"\nProcessing {idx + 1}/{len(csv_parts)} file: {csv_part} \n")
        df = pd.read_csv(csv_part)
        split_a_part(df, save_path=save_path, is_append=is_append, is_compressed=is_compressed)d

save_path = r'/media/gpu/Seagate/SC_weekly_patterns_2020_split'  
csv_dir = r'/media/gpu/Seagate/SC_weekly_patterns_20220101'
csv_parts = glob.glob(os.path.join(csv_dir, "SC-part*.csv.gz"))

split_all_part(csv_parts, save_path=save_path, is_append=True, is_compressed=True)

Processing 1/6 file: /media/gpu/Seagate/SC_weekly_patterns_20220101/SC-part1.csv.gz 

500 ('45', '450130', '2020-10-12_2020-10-19') /media/gpu/Seagate/SC_weekly_patterns_2020_split/45/450130
1000 ('45', '450259', '2020-03-09_2020-03-16') /media/gpu/Seagate/SC_weekly_patterns_2020_split/45/450259
1500 ('45', '450379', '2021-03-22_2021-03-29') /media/gpu/Seagate/SC_weekly_patterns_2020_split/45/450379
2000 ('45', '450499', '2021-03-01_2021-03-08') /media/gpu/Seagate/SC_weekly_patterns_2020_split/45/450499
2500 ('45', '450619', '2020-06-15_2020-06-22') /media/gpu/Seagate/SC_weekly_patterns_2020_split/45/450619
3000 ('45', '450719', '2021-11-08_2021-11-15') /media/gpu/Seagate/SC_weekly_patterns_2020_split/45/450719
3500 ('45', '450850', '2020-07-20_2020-07-27') /media/gpu/Seagate/SC_weekly_patterns_2020_split/45/450850
Processing 2/6 file: /media/gpu/Seagate/SC_weekly_patterns_20220101/SC-part2.csv.gz 

500 ('45', '450130', '2020-10-12_2020-10-19') /media/gpu/Seagate/SC_weekly_patterns_202

In [69]:
COLS_FROM_CORE_PLACES = ['safegraph_place_id', 'parent_safegraph_place_id', 'location_name', 'safegraph_brand_ids', 
                         'brands', 'top_category', 'sub_category', 'naics_code', 'latitude', 'longitude', 'street_address',
                         'city', 'region', 'postal_code', 'open_hours']

COLS_FROM_CORE_PLACES_FOOTPRINT = ['placekey', 'safegraph_place_id', 'polygon_class', 'area_square_feet', 'min_area']

COLS_FROM_WEEKLY_PATTERNS = ['sg_wp__parent_placekey', 'sg_wp__poi_cbg', 'sg_wp__visitor_home_cbgs', 'sg_wp__visitor_daytime_cbgs', 
                             'sg_wp__visitor_country_of_origin', 'sg_wp__distance_from_home', 'sg_wp__median_dwell', 'sg_wp__bucketed_dwell_times']

## Load the core_POI

In [62]:
core_poi_df = helper.load_core_places_data(COLS_FROM_CORE_PLACES)

Loading /media/gpu/easystore/Safegraph/Core Places US (Nov 2020 - Present)/core_poi/2021/06/05/00/core_poi-part1.csv.gz
Loading /media/gpu/easystore/Safegraph/Core Places US (Nov 2020 - Present)/core_poi/2021/06/05/00/core_poi-part2.csv.gz
Loading /media/gpu/easystore/Safegraph/Core Places US (Nov 2020 - Present)/core_poi/2021/06/05/00/core_poi-part3.csv.gz
Loading /media/gpu/easystore/Safegraph/Core Places US (Nov 2020 - Present)/core_poi/2021/06/05/00/core_poi-part4.csv.gz
Loading /media/gpu/easystore/Safegraph/Core Places US (Nov 2020 - Present)/core_poi/2021/06/05/00/core_poi-part5.csv.gz
Loading /media/gpu/easystore/Safegraph/Core Places US (Nov 2020 - Present)/core_poi/2021/06/05/00/core_poi-part6.csv.gz
Loading core places info for 6462532 POIs


## Load the area column to the CORE_POIs.

In [67]:
def load_core_places_footprint_data(cols_to_keep):
    FOOTPRINT_FILE = r'/media/gpu/easystore/Safegraph/Core Places US (Nov 2020 - Present)/core_poi/core_POI_area.csv' 
    area_csv = os.path.join(FOOTPRINT_FILE)
    print('Loading', area_csv)
    df = load_csv_possibly_with_dask(area_csv, usecols=cols_to_keep, use_dask=False)
    df = df.set_index('safegraph_place_id')
    print('Loaded core places footprint data for %d POIs' % len(df))
    return df

# footprint_df = load_core_places_footprint_data(COLS_FROM_CORE_PLACES_FOOTPRINT)
footprint_df = load_core_places_footprint_data(None)

Loading /media/gpu/easystore/Safegraph/Core Places US (Nov 2020 - Present)/core_poi/core_POI_area.csv
Loaded core places footprint data for 5941711 POIs


In [68]:
footprint_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5941711 entries, sg:000000c8f55d451ca1ae98bd057785a7 to sg:fffffc1c8810425e8197760fc5328e89
Data columns (total 28 columns):
 #   Column                Dtype  
---  ------                -----  
 0   location_name         object 
 1   polygon_class         object 
 2   is_synthetic          object 
 3   includes_parking_lot  object 
 4   iso_country_code      object 
 5   area_square_feet      float64
 6   sg_g.paren            object 
 7   sg_g.locat            object 
 8   sg_g.brand            object 
 9   sg_g.latit            float64
 10  sg_g.longi            float64
 11  sg_g.stree            object 
 12  sg_g.city             object 
 13  sg_g.regio            object 
 14  sg_g.posta            float64
 15  sg_g.iso_c            object 
 16  sg_g.polyg            object 
 17  sg_g.inclu            float64
 18  sg_g.is_sy            float64
 19  sg_g.build            float64
 20  sg_g.enclo            float64
 21  area_meter          

In [65]:
footprint_df['area_square_feet'] = footprint_df['min_area']
footprint_df

,polygon_class,area_square_feet,min_area
safegraph_place_id,,,
sg:000000c8f55d451ca1ae98bd057785a7,OWNED_POLYGON,239.658301,239.658301
sg:000008c003ab41d9b6a8d1e58e4992ea,OWNED_POLYGON,95.644273,95.644273
sg:0000093d8e7741e8a2a7ab13d432aaa2,NaN,376.543914,376.543914
sg:00000a99a12a45cd80342b9f881eced6,NaN,1236.058087,1236.058087
sg:00000b5b33aa4d27a98b815f163ccd3b,OWNED_POLYGON,181.531029,181.531029
...,...,...,...
sg:fffff90467044c7789e1594fdae67934,NaN,204.502240,204.502240
sg:fffff977b3324b7eac27c39b3036ff0f,NaN,668.063234,668.063234
sg:fffffafcfc1246349f856586a4f92d74,OWNED_POLYGON,3358.956378,3358.956378


In [66]:
footprint_df['area_square_feet'].isna().sum()

0

In [23]:
df['date_range_start'].max()

'2021-11-22'

In [14]:
df2 = pd.read_csv(csv_parts[2])

df2

,date_range_start,date_range_end,placekey,sg_wp__parent_placekey,sg_wp__location_name,sg_wp__street_address,sg_wp__city,sg_wp__region,sg_wp__postal_code,sg_wp__iso_country_code,...,sg_wp__visitor_home_cbgs,sg_wp__visitor_home_aggregation,sg_wp__visitor_daytime_cbgs,sg_wp__visitor_country_of_origin,sg_wp__distance_from_home,sg_wp__median_dwell,sg_wp__bucketed_dwell_times,sg_wp__related_same_day_brand,sg_wp__related_same_week_brand,sg_wp__device_type
0,2021-06-07,2021-06-14,222-222@645-wrz-nqz,NaN,Harrington Vision Center,181 W Cheves St,Florence,SC,29501,US,...,"{""450910614011"":5,""450310112004"":5,""4504100160...","{""45041001400"":5,""45041000600"":4,""45041000800""...","{""450410014004"":4,""450410007001"":4,""4504100150...","{""US"":59}",8749.0,20.0,"{""<5"":4,""5-10"":21,""11-20"":11,""21-60"":15,""61-12...","{""Walmart"":15,""McDonald's"":9,""CVS"":8,""Sonic"":6...","{""Chick-fil-A"":34,""Walmart"":32,""McDonald's"":25...","{""android"":21,""ios"":34}"
1,2021-06-07,2021-06-14,222-222@645-x2g-7yv,223-222@645-x2g-7yv,Big Lots Stores,2528 David H McLeod Blvd,Florence,SC,29501,US,...,"{""450410014005"":6,""450410002022"":5,""4504100020...","{""45041001504"":8,""45089970200"":7,""45041000202""...","{""450410015041"":6,""450410015051"":5,""4508500020...","{""US"":130}",22772.0,22.0,"{""<5"":3,""5-10"":28,""11-20"":26,""21-60"":72,""61-12...","{""Walmart"":22,""Shell Oil"":13,""Lowe's"":9,""Sam's...","{""Walmart"":61,""Shell Oil"":34,""Exxon Mobil"":32,...","{""android"":62,""ios"":71}"
2,2021-06-07,2021-06-14,222-222@645-zw8-9xq,NaN,Shell Oil,5167 Highway 38 N,Bennettsville,SC,29512,US,...,"{""391199115001"":7,""450699601003"":6,""4504392050...","{""45069960100"":9,""39035123602"":7,""37081017100""...","{""540739622003"":5,""370570617033"":5,""3904900721...","{""US"":210}",234707.0,8.0,"{""<5"":21,""5-10"":161,""11-20"":31,""21-60"":11,""61-...","{""McDonald's"":16,""Walmart"":15,""Exxon Mobil"":10...","{""Walmart"":47,""McDonald's"":43,""Exxon Mobil"":23...","{""android"":92,""ios"":131}"
3,2021-06-07,2021-06-14,222-222@646-5ny-wc5,NaN,Comprehensive Dentistry,5879 Highway 707,Socastee,SC,29588,US,...,"{""450510601021"":4,""450510516042"":4,""4505105160...","{""45051051604"":4,""45051051603"":4,""45051060102""...","{""450510503031"":4,""450510602032"":4}","{""US"":20}",5896.0,24.0,"{""<5"":2,""5-10"":8,""11-20"":4,""21-60"":5,""61-120"":...","{""Speedway"":11,""Walmart"":7,""Sonic"":4,""Cellular...","{""McDonald's"":46,""Walmart"":33,""Speedway"":25,""C...","{""android"":13,""ios"":12}"
4,2021-06-07,2021-06-14,222-222@646-5pp-y5f,NaN,Black Diamond Detailing,6001 S Kings Hwy Ste 5427,Surfside Beach,SC,29575,US,...,"{""371290117051"":4,""371570410011"":4,""2802595010...","{""45051051700"":4,""37179020401"":4,""37105030601""...","{""450510514032"":4,""370710307002"":4,""3715704030...","{""US"":57}",299484.0,28.0,"{""<5"":3,""5-10"":30,""11-20"":11,""21-60"":31,""61-12...","{""Walmart"":18,""McDonald's"":7,""Murphy USA"":6,""C...","{""Walmart"":70,""McDonald's"":38,""Chick-fil-A"":32...","{""android"":18,""ios"":41}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193569,2021-03-29,2021-04-05,zzz-222@8gg-szq-nt9,NaN,University Park,101 Brookside Cir,Greenville,SC,29609,US,...,"{""450830230014"":4,""450450026111"":4,""4504500210...","{""45045002702"":4}","{""450450026111"":4,""450450011014"":4}","{""US"":16}",9271.0,75.5,"{""<5"":3,""5-10"":2,""11-20"":0,""21-60"":8,""61-120"":...","{""Walmart"":13,""Chick-fil-A"":13,""Waffle House"":...","{""Chick-fil-A"":29,""Zaxby's"":24,""Waffle House"":...","{""android"":5,""ios"":13}"
1193570,2021-03-29,2021-04-05,zzz-222@8gg-vrv-kxq,NaN,Jaycee Park,Edgewood Ave & Shorecrest Dr,Clemson,SC,29631,US,...,"{""450770112051"":7,""450770112032"":4,""4500701060...","{""45063021203"":4,""45077010300"":4,""45077010804""...","{""131950202002"":5,""450730302002"":4,""4507701110...","{""US"":40}",7331.0,27.0,"{""<5"":3,""5-10"":12,""11-20"":5,""21-60"":19,""61-120

In [15]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1193574 entries, 0 to 1193573
Data columns (total 29 columns):
 #   Column                            Non-Null Count    Dtype  
---  ------                            --------------    -----  
 0   date_range_start                  1193574 non-null  object 
 1   date_range_end                    1193574 non-null  object 
 2   placekey                          1193574 non-null  object 
 3   sg_wp__parent_placekey            112839 non-null   object 
 4   sg_wp__location_name              1193574 non-null  object 
 5   sg_wp__street_address             1193574 non-null  object 
 6   sg_wp__city                       1193574 non-null  object 
 7   sg_wp__region                     1193574 non-null  object 
 8   sg_wp__postal_code                1193574 non-null  int64  
 9   sg_wp__iso_country_code           1193574 non-null  object 
 10  sg_wp__safegraph_brand_ids        296203 non-null   object 
 11  sg_wp__brands                     296